In [1]:
# %% import package to use
import datatable
import pandas as pd
from IPython.display import display
import numpy as np
import os
from sklearn.metrics import accuracy_score
from imblearn.metrics import sensitivity_specificity_support
import matplotlib.pyplot as plt

from sklearn.metrics import log_loss
import xgboost as xgb
pd.set_option('display.max_columns', None)

from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import roc_auc_score

import seaborn as sns
from statannot import add_stat_annotation

In [2]:
import pickle

#### XGBoost
with open('../asthma_xg_classifier_subset_feature_selection.pickle', 'rb') as file_nm:
    results = pickle.load(file_nm)
    
df_results_xg = pd.DataFrame()
for condition in results:
    df_results_xg = pd.concat((df_results_xg, pd.DataFrame(results[condition])), axis=0)
    
df_results_xg = df_results_xg.reset_index(drop=True)

# xg_best_param = df_results_xg[df_results_xg['mean_mse'] == df_results_xg['mean_mse'].min()][['max_depth', 'learning_rate', 'lambda', 'gamma', 'scores', 'mse_loss']].reset_index(drop=True)

In [3]:
df_results_xg['max_acc_by_feature'] = df_results_xg.groupby(['feature'])['mean_score'].transform(lambda x: x.max())
df_results_xg['min_loss_by_feature'] = df_results_xg.groupby(['feature'])['mean_loss'].transform(lambda x: x.min())#.query('mean_score == mean_score.max()', engine='python')

KeyError: 'Column not found: mean_loss'

In [4]:
best_metric_accuracy = df_results_xg.query('mean_score == max_acc_by_feature')
best_metric_loss = df_results_xg.query('mean_score == min_loss_by_feature')

UndefinedVariableError: name 'min_loss_by_feature' is not defined

In [ ]:
plt.figure(figsize=(10,10))
x = 'feature'
y = 'scores'
order = ['sp_eosinophil', 'Bl_eos_count', 'maxFall_FEV1_percent', 'FEF2575_afterMBPT']
ax = sns.barplot(data=best_metric_accuracy, x=x, y=y, order=order)

ax, test_results = add_stat_annotation(ax, data=best_metric_accuracy, x=x, y=y, order=order,
                                   box_pairs=[('sp_eosinophil', 'Bl_eos_count'), 
                                              ('sp_eosinophil', 'maxFall_FEV1_percent'), 
                                              ('sp_eosinophil', 'FEF2575_afterMBPT'), 
                                              ('Bl_eos_count', 'maxFall_FEV1_percent'), 
                                              ('Bl_eos_count', 'FEF2575_afterMBPT'), 
                                              ('maxFall_FEV1_percent', 'FEF2575_afterMBPT')],
                                   test='t-test_ind', text_format='star', loc='outside', verbose=2)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
plt.ylim(0, 110)
plt.xlabel("")
plt.ylabel('Accuracy', fontsize=20)
sns.despine()

In [ ]:
plt.figure(figsize=(10,10))
x = 'feature'
y = 'scores'
order = ['sp_eosinophil', 'Bl_eos_count', 'maxFall_FEV1_percent', 'FEF2575_afterMBPT']
ax = sns.barplot(data=best_metric_loss, x=x, y=y, order=order)

ax, test_results = add_stat_annotation(ax, data=best_metric_loss, x=x, y=y, order=order,
                                   box_pairs=[('sp_eosinophil', 'Bl_eos_count'), 
                                              ('sp_eosinophil', 'maxFall_FEV1_percent'), 
                                              ('sp_eosinophil', 'FEF2575_afterMBPT'), 
                                              ('Bl_eos_count', 'maxFall_FEV1_percent'), 
                                              ('Bl_eos_count', 'FEF2575_afterMBPT'), 
                                              ('maxFall_FEV1_percent', 'FEF2575_afterMBPT')],
                                   test='t-test_ind', text_format='star', loc='outside', verbose=2)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
plt.ylim(0, 110)
plt.xlabel("")
plt.ylabel('Loss', fontsize=20)
sns.despine()